In [ ]:
# Why use functional API? To create more complex models and to provide more functionality through them.

In [1]:
# there are many datasets availabale in tensorflow too which we can load using tensorflow_datasets
import tensorflow as tf # for models
import numpy as np # for mathematical computations
import matplotlib.pyplot as plt # for visualization
import tensorflow_datasets as tfds # for the malaria dataset
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, InputLayer, Flatten, BatchNormalization, Input, Layer
from google.colab import drive
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, Accuracy

#### **DATA PREPARATION**

In [2]:
# We would create the same model as malaria detection with the help of functional API
dataset, dataset_info = tfds.load(
    'malaria',
    with_info = True,
    as_supervised = True,
    shuffle_files = True,
    split = ['train']
)
dataset[0] # Note : we can apply take on 'Dataset' datatypes only.

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/incomplete.TWAXLO_1.0.0/malaria-train.tfrecord*...:   0%|         …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [3]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
  DATASET_SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_test_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE)+int(VAL_RATIO*DATASET_SIZE))

  return train_dataset,val_test_dataset,test_dataset

In [4]:
train_dataset, val_dataset, test_dataset = splits(dataset[0],0.8,0.1,0.1)
print(list(train_dataset.take(1).as_numpy_iterator()),list(val_dataset.take(1).as_numpy_iterator()),list(test_dataset.take(1).as_numpy_iterator()))

[(array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), np.int64(1))] [(array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0,

#### **DATA PREPROCESSING**

In [5]:
# Resizing the images and rescaling (Normalizing)
IM_SIZE = 224
def resize_rescale(image,label):
  return tf.image.resize(image, (IM_SIZE, IM_SIZE))/255.0, label

In [6]:
train_dataset = train_dataset.map(resize_rescale) # This is used for applying the mentioned function on every value of dataset like apply
# in pandas

In [7]:
val_dataset = val_dataset.map(resize_rescale)

In [8]:
test_dataset = test_dataset.map(resize_rescale)

In [9]:
train_dataset

<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [10]:
train_dataset = train_dataset.shuffle(buffer_size = 512,reshuffle_each_iteration = True).batch(32).prefetch(tf.data.AUTOTUNE)

In [11]:
val_dataset = val_dataset.shuffle(buffer_size = 512,reshuffle_each_iteration = True).batch(32).prefetch(tf.data.AUTOTUNE)

In [12]:
test_dataset = test_dataset.shuffle(buffer_size = 512,reshuffle_each_iteration = True).batch(32).prefetch(tf.data.AUTOTUNE)

#### **MODEL PREPARATION**

In [ ]:
model = tf.keras.Sequential([
    InputLayer(shape = (IM_SIZE,IM_SIZE,3)),

    Conv2D(filters = 6,kernel_size = 3,strides = 1,padding = 'valid',activation = 'relu'),
    BatchNormalization(),
    MaxPool2D(pool_size = 2,strides = 2),

    Conv2D(filters = 16,kernel_size = 3,strides = 1,padding = 'valid',activation = 'relu'),
    BatchNormalization(),
    MaxPool2D(pool_size = 2,strides = 2),

    Flatten(),

    Dense(100,activation = 'relu'),
    BatchNormalization(),
    Dense(10,activation = 'relu'),
    BatchNormalization(),
    Dense(1,activation = 'sigmoid')
])
model.summary() # This is made by sequential model which we would not use here.

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

In [ ]:
# Creating the same above model with functional API
func_input = Input(shape = (IM_SIZE,IM_SIZE,3),name = 'Input Image') # works just like the InputLayer which prefetchs the shape of the input.

x = Conv2D(filters = 6,kernel_size = 3,strides = 1,padding = 'valid',activation = 'relu')(func_input)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size = 2,strides = 2)(x)

x = Conv2D(filters = 16,kernel_size = 3,strides = 1,padding = 'valid',activation = 'relu')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size = 2,strides = 2)(x) # upto this point the part of the model can be called feature extraction part as the model is
# extracting important features upto this point.

x = Flatten()(x)

x = Dense(100,activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dense(10,activation = 'relu')(x)
x = BatchNormalization()(x) # and this whole part of Dense Layers is responsible for correctly classifying the input to the corresponding label

func_output = Dense(1,activation = 'sigmoid')(x) # basically we are taking the input given.The intermediate output
# (which are passed as inputs to the subsequent layers) is being passed as a functional argument here to each of the layers.

LeNet_model = Model(func_input, func_output) # Basically we named all the above process as LeNet_model that's it.
LeNet_model.summary() # The summary would be same as that of the above model starting from 2nd position as in sequential model inputlayer
# is not shown by default.

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input Image (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

In [ ]:
# We can compile and train the above model just as we compiled the "model". So, there is no point in repeating the same process without
# much meaning to it.

In [ ]:
# But isn't the model we just made through functional API looking odd? We wrote x so many times in it why don't we create seperate models for
# the feature extraction and the classification and then later combine them? Yes, it's possible.
func_input = Input(shape = (IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = Conv2D(filters = 6,kernel_size = 3,strides = 1,padding = 'valid',activation = 'relu')(func_input)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size = 2,strides = 2)(x)

x = Conv2D(filters = 16,kernel_size = 3,strides = 1,padding = 'valid',activation = 'relu')(x)
x = BatchNormalization()(x)
output = MaxPool2D(pool_size = 2,strides = 2)(x)

Feature_extractor = Model(func_input,output)
Feature_extractor.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input Image (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,136 (4.44 KB)

 Trainable params: 1,092 (4.27 KB)

 Non-trainable params: 44 (176.00 B)

In [ ]:
# Now,
func_input = Input(shape = (IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = Feature_extractor(func_input) # Notice how we are 'calling' a model like a function, thanks to tensorflow.

x = Flatten()(x)

x = Dense(100,activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dense(10,activation = 'relu')(x)
x = BatchNormalization()(x)
func_output = Dense(1,activation = 'sigmoid')(x)

LeNet_Model = Model(func_input,func_output,name = 'LeNet_Model')
LeNet_Model.summary() # This works as fine as the previous one, as we observe that there are same number of params here too.

Model: "LeNet_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input Image (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional (Functional)         │ (None, 54, 54, 16)     │         1,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

In [ ]:
# Also we can mix the functional API and Sequential API to make a model.
# Let's declare the classification model part with Sequential API.
Classifier = tf.keras.Sequential([
    InputLayer(shape = (46656,)),

    Dense(100, activation = 'relu'),
    BatchNormalization(),
    Dense(10, activation = 'relu'),
    BatchNormalization(),
    Dense(1, activation = 'sigmoid')
])
Classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,667,161 (17.80 MB)

 Trainable params: 4,666,941 (17.80 MB)

 Non-trainable params: 220 (880.00 B)

In [ ]:
# Now, combining the sequential classifier and functional extractor,
func_input = Input(shape = (IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = Feature_extractor(func_input)

x = Flatten()(x)

func_output = Classifier(x)
LeNet_model = Model(func_input,func_output,name = 'LeNet_model')
LeNet_model.summary() # This model is much more abstracted than any other previous models.
# And as we can notice from below that the model has both functional as well as sequential layers in it.

Model: "LeNet_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input Image (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional (Functional)         │ (None, 54, 54, 16)     │         1,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 1)              │     4,667,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

#### **MODEL SUBCLASSING**

In [61]:
# Before starting know one thing that for all layers like Dense, Conv2D there is a base class called Layer() or it is the parent class
# of all of them so whenever you make your custom class make sure you inherit it from the 'Layer' class.

class FeatureExtractor(Layer):
  def __init__(self): # note we can pass the kernel size and other parameters while making the object itself but here keeping things simple
    super().__init__() # we used it so that we can also use the base class' (Layer's) constructor.

    self.conv_1 = Conv2D(filters = 6,kernel_size = 3,strides = 1,padding = 'valid',activation = 'relu')
    self.batch_1 = BatchNormalization()
    self.pool_1 = MaxPool2D(pool_size = 2,strides = 2)

    self.conv_2 = Conv2D(filters = 16,kernel_size = 3,strides = 1,padding = 'valid',activation = 'relu')
    self.batch_2 = BatchNormalization()
    self.pool_2 = MaxPool2D(pool_size = 2,strides = 2)

  def call(self,x):
    x = self.conv_1(x)
    x = self.batch_1(x)
    x = self.pool_1(x)

    x = self.conv_2(x)
    x = self.batch_2(x)
    x = self.pool_2(x)

    return x

Extract_feautures = FeatureExtractor() # The object gets successfully created.

In [ ]:
# Trying to make the same model again,
func_input = Input(shape = (IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = Extract_feautures.call(func_input) # note: in the summary, we get to know that this thing is not functional API thing.

x = Flatten()(x)

func_output = Classifier(x)
LeNet_model = Model(func_input,func_output,name = 'LeNet_model')
LeNet_model.summary() # as we can see we still got the same model hence this worked out correctly too.

NameError: name 'Classifier' is not defined

In [62]:
# Let's just make a model now using above thing the only change would be that the base class would be a MODEL.

class LeNetModel(Model):
  def __init__(self):
    super().__init__()

    self.features_extract = FeatureExtractor()
    self.flatten = Flatten()

    self.Dense_1 = Dense(100,activation = 'relu')
    self.batch_1 = BatchNormalization()

    self.Dense_2 = Dense(10,activation = 'relu')
    self.batch_2 = BatchNormalization()

    self.Dense_3 = Dense(1,activation = 'sigmoid')

  def call(self,x):

    x = self.features_extract(x)
    x = self.flatten(x)
    x = self.Dense_1(x)
    x = self.batch_1(x)
    x = self.Dense_2(x)
    x = self.batch_2(x)
    x = self.Dense_3(x)

    return x

LeNet_Model_sub = LeNetModel()
LeNet_Model_sub.call(tf.zeros([1,224,224,3]))
LeNet_Model_sub.summary()

Model: "le_net_model_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ feature_extractor_23            │ ?                      │         1,136 │
│ (FeatureExtractor)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (1, 46656)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (1, 100)               │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_72          │ (1, 100)               │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (1, 10)                │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_73          │ (1, 10)                │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (1, 1)                 │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

#### **MODEL TRAINING**

In [63]:
LeNet_Model_sub.compile(
    optimizer = Adam(learning_rate = 0.001),
    loss = BinaryCrossentropy(),
    metrics = ['accuracy']
)

In [64]:
history = LeNet_Model_sub.fit(train_dataset,validation_data = val_dataset,epochs = 6,verbose = 1) # paused because it had attained much higher
# val accuracy already and further it, it would start to overfit.

Epoch 1/6
689/689 ━━━━━━━━━━━━━━━━━━━━ 44s 58ms/step - accuracy: 0.7748 - loss: 0.4507 - val_accuracy: 0.9423 - val_loss: 0.1854
Epoch 2/6
689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 56ms/step - accuracy: 0.9396 - loss: 0.1834 - val_accuracy: 0.9437 - val_loss: 0.1787
Epoch 3/6
689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 56ms/step - accuracy: 0.9479 - loss: 0.1450 - val_accuracy: 0.9546 - val_loss: 0.1631
Epoch 4/6
689/689 ━━━━━━━━━━━━━━━━━━━━ 47s 67ms/step - accuracy: 0.9614 - loss: 0.0999 - val_accuracy: 0.9456 - val_loss: 0.1939
Epoch 5/6
689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 56ms/step - accuracy: 0.9761 - loss: 0.0632 - val_accuracy: 0.9354 - val_loss: 0.2522
Epoch 6/6
689/689 ━━━━━━━━━━━━━━━━━━━━ 38s 55ms/step - accuracy: 0.9847 - loss: 0.0395 - val_accuracy: 0.9285 - val_loss: 0.2652


In [65]:
LeNet_Model_sub.evaluate(test_dataset)

87/87 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step - accuracy: 0.9270 - loss: 0.2659


[0.2835952937602997, 0.9281827807426453]

In [66]:
# as we can see the sequential and functional API both can achieve much higher accuracy hence it is convenient to use both

####**CUSTOM LAYER**

In [21]:
# We would create the Dense Layer here from scratch by the tensorflow's custom layers
class KrishDense(Layer):
  def __init__(self,hidden_units,activation):    # <-- Note: You have to take care of the layer's parameters too as they can not be neglected.
    super().__init__()
    self.neurons = hidden_units
    self.activation = activation

  def build(self,input_features):
    self.w = self.add_weight(              # <-- Also the add_weight is inherited through layer in our KrishDense
        (input_features[-1],self.neurons),
        trainable = True,
        initializer = "random_normal"
    )
    self.b = self.add_weight(
        (self.neurons,),
        trainable = True,
        initializer = "random_normal"
    )

  def call(self,input_features):
    if (self.activation == 'relu'):
      return tf.nn.relu(tf.matmul(input_features,self.w) + self.b)

    elif(self.activation == 'sigmoid'):
      return tf.math.sigmoid(tf.matmul(input_features,self.w) + self.b)

    else:
      return tf.matmul(input_features,self.w) + self.b

In [27]:
fun_model = tf.keras.Sequential([
    InputLayer(shape = (112,112,3)),
    Flatten(),
    KrishDense(100,activation = 'relu'),
    KrishDense(50,activation = 'relu'),
    KrishDense(1,activation = 'sigmoid')
])
fun_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_2 (Flatten)             │ (None, 37632)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ krish_dense_18 (KrishDense)     │ (None, 100)            │     3,763,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ krish_dense_19 (KrishDense)     │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ krish_dense_20 (KrishDense)     │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,768,401 (14.38 MB)

 Trainable params: 3,768,401 (14.38 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# To build a functional API model,
func_input = Input(shape = (IM_SIZE,IM_SIZE,3),name = 'Input Image')

x = Flatten()(func_input)

func_output = KrishDense(activation = 'relu',hidden_units = 2)(x) # Note: the syntax

fun_func_model = Model(func_input,func_output,name = 'Fun_Model')
fun_func_model.summary()

Model: "Fun_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input Image (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 150528)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ krish_dense_21 (KrishDense)     │ (None, 2)              │       301,058 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 301,058 (1.15 MB)

 Trainable params: 301,058 (1.15 MB)

 Non-trainable params: 0 (0.00 B)